In [ ]:
# скачиваем файл
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1rwQRsjsUgJjFYJH5OhkzOC7PeezZhdEl' -O heart.csv

In [ ]:
# устанавливаем нужную версию библиотеки sklearn, чтобы ответы сошлись
!pip install scikit-learn==0.22.1

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
plt.rcParams['figure.figsize'] = (11, 6.5)

В этом задании вы будете работать с [данными о пациентах, у части которых есть заболевание сердца](https://www.kaggle.com/ronitf/heart-disease-uci). Вам нужно построить модель, классифицирующую пациентов на больных этим заболеванием и тех, у кого его нет, а также проанализировать результаты.

In [3]:
data = pd.read_csv('heart.csv')
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


Разделим данную выборку на обучающую и тестовую части в отношении 3:1.

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    data.drop('target', axis=1),
    data['target'],
    test_size=0.25,
    random_state=13
)
X_train.shape, X_test.shape

((227, 13), (76, 13))

1. Обучите решающее дерево из `sklearn` (`sklearn.tree.DecisionTreeClassifier`) без ограничения на максимальную глубину (`max_depth=None`). В качестве сида поставьте `random_state=13`.

   Подробнее о параметрах можно почитать в [документации](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html).

**Найдите долю правильных ответов полученного алгоритма на обучающей выборке.**

In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

clf = DecisionTreeClassifier(random_state=13)
clf.fit(X_train, y_train)

predicted = clf.predict(X_train)
f'train accuracy = {accuracy_score(y_train, predicted)}'

'train accuracy = 1.0'

2. Теперь найдите долю правильных ответов полученного алгоритма на тестовой выборке.

**Заметно ли переобучение?**

- Да, максимальная точность на трейне и слабая на тесте -> переобучение.

In [6]:
predicted = clf.predict(X_test)
f'test accuracy = {accuracy_score(y_test, predicted):.3f}'

'test accuracy = 0.671'

3. Подберите с помощью кросс-валидации оптимальные гиперпараметры алгоритма. Выбирайте из следующих наборов:


- `max_depth`: [3, 4, 5, 6, 7, 8, 9, 10, None]
- `max_features`: ['auto', 'log2', None]
- `min_samples_leaf`: range(1, 10)
- `min_samples_split`: range(2, 10)
- `criterion`: ['gini', 'entropy']

    В этом вам поможет метод `sklearn.model_selection.GridSearchCV`. Подробнее о том, какие параметры и методы в нем используются, и о примерах работы с ним можно прочитать в [документации](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).
    
    1) Создайте решающее дерево - не забудьте поставить `random_state=13`.
    
    2) Задайте `param_grid` - сетку (словарь) гиперпараметров и их значений, по которой будет проходить метод.
    
    3) Вызовите метод `GridSearchCV` - в качестве параметра `estimator` задайте решающее дерево из первого шага, `param_grid` - сетку из второго. Задайте параметр `cv=5`, чтобы кросс-валидация проходила по 5 фолдам. Также задайте параметр `scoring='accuracy'`, чтобы оценка качества моделей на кросс-валидации проходила с помощью доли правильных ответов. Запустите метод на обучающей выборке с помощью `fit`. 
    
    4) Выведите лучшие параметры с помощью атрибута `best_params_`.
    
    Какое значение глубины дерева получилось оптимальным?

In [7]:
from sklearn.model_selection import GridSearchCV

dtclf = DecisionTreeClassifier(random_state=13)
parameters = {
    'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, None],
    'max_features': ['auto', 'log2', None],
    'min_samples_leaf': range(1, 10),
    'min_samples_split': range(2, 10),
    'criterion': ['gini', 'entropy']
}
clf = GridSearchCV(dtclf, parameters, scoring='accuracy', cv=5)

clf.fit(X_train, y_train)

clf.best_params_

{'criterion': 'gini',
 'max_depth': 9,
 'max_features': 'auto',
 'min_samples_leaf': 3,
 'min_samples_split': 9}

4. Какое лучшее усредненное значение доли правильных ответов получилось на кросс-валидации (для оптимальных значений гиперпараметров)? Вам поможет атрибут `best_score_`.

In [8]:
f'для оптимальных параметров лучшее среднее accuracy на всех фолдах = {clf.best_score_:.3f}'

'для оптимальных параметров лучшее среднее accuracy на всех фолдах = 0.832'

5. Найдите долю правильных ответов решающего дерева с подобранными оптимальными значениями гиперпараметров на обучающей выборке.

In [9]:
best_params = clf.best_params_
best_params['random_state'] = 13

best_clf = DecisionTreeClassifier(
    **best_params
)
best_clf.fit(X_train, y_train)

predicted = best_clf.predict(X_train)

f'train accuracy = {accuracy_score(y_train, predicted):.3f}'

'train accuracy = 0.907'

6. Найдите долю правильных ответов решающего дерева с подобранными оптимальными значениями гиперпараметров на тестовой выборке.

**Уменьшилось ли переобучение?** 
- Да. На обучающей выборке accuracy стала меньше, а на тестовой стала лучше.

In [10]:
best_params = clf.best_params_
best_params['random_state'] = 13

best_clf = DecisionTreeClassifier(
    **best_params
)
best_clf.fit(X_train, y_train)

predicted = best_clf.predict(X_test)

f'test accuracy = {accuracy_score(y_test, predicted):.3f}'

'test accuracy = 0.737'

7. Решающее дерево позволяет предсказывать не только классы, но и вероятности классов - с помощью метода `predict_proba`. Посмотрите на вероятности классов полученного решающего дерева и посчитайте значение AUC-ROC.

In [11]:
from sklearn.metrics import roc_auc_score

predicted_proba = best_clf.predict_proba(X_test)[:, 1]
f'AUC-ROC = {roc_auc_score(y_test, predicted_proba):.3f}'

'AUC-ROC = 0.772'

8. Какой признак является самым важным по мнению полученного решающего дерева? Чтобы это проверить, вам поможет атрибут `feature_importances_`.

In [12]:
feature_idx = np.argmax(best_clf.feature_importances_)
f'наиболее важный признак = {data.columns[feature_idx]}'

'наиболее важный признак = slope'